# Modelo de detección de lenguaje tóxico

En este notebook se explorarán varias estrategias para generar un modelo de detección de lenguaje tóxico con modelos de MAchine Learning.

In [1]:
import time 
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import joblib
import warnings
warnings.filterwarnings("ignore")

## Dataset

El dataset utilizado será el de la competición de Kaggle traducido a catellano, además se han unificado todas las categorías referentes al lenguaje tóxico de forma que los comentarios se clasifican en toxico/no_toxico.

Este dataset consta de 159571 textos de los cuáles 16225 son tóxicos. Por lo que se trata de un dataset muy desbalanceado y deberemos tenerlo en cuenta durante el entrenamiento.

In [2]:
df = pd.read_csv(r'..\data\internal\toxicity.csv')

In [3]:
df.head()

,spanish_text,label
0,Explicación ¿Por qué las ediciones realizadas ...,no_toxico
1,D'aww! Coincide con este color de fondo con el...,no_toxico
2,"Hey hombre, realmente no estoy tratando de edi...",no_toxico
3,""" Más no puedo hacer ninguna sugerencia real s...",no_toxico
4,"Usted, señor, es mi héroe. ¿Alguna posibilidad...",no_toxico


In [4]:
len(df)

159571

In [5]:
df.groupby("label").count()

,spanish_text
label,
no_toxico,143346
toxico,16225


## Elección de la pipeline

En primer lugar, utilizamos el subconjunto formado por los 10000 primeros textos para probar distintas pipelines.

Compararemos las métricas obtenidas y los tiempos de entrenamiento y predicción para seleccionar la más conveniente.

In [6]:
df[:10000]

,spanish_text,label
0,Explicación ¿Por qué las ediciones realizadas ...,no_toxico
1,D'aww! Coincide con este color de fondo con el...,no_toxico
2,"Hey hombre, realmente no estoy tratando de edi...",no_toxico
3,""" Más no puedo hacer ninguna sugerencia real s...",no_toxico
4,"Usted, señor, es mi héroe. ¿Alguna posibilidad...",no_toxico
...,...,...
9995,Los números pueden ser listados por separado a...,no_toxico
9996,"A esos dos les encanta estar en desacuerdo, ¿v...",no_toxico
9997,"""He cambiado """"Lance Thomas"""" por """"Lance Thom...",no_toxico
9998,"Si bien los tribunales son, por supuesto, part...",no_toxico


In [7]:
df[:10000].groupby("label").count()

,spanish_text
label,
no_toxico,8970
toxico,1030


In [8]:
x_train, x_test, y_train, y_test = train_test_split(list(df[:10000]['spanish_text']), df[:10000]['label'])

In [9]:
models = {}
results = []

def get_name(o):
    return o.__class__.__name__

for lang in [CountVectorizer(), TfidfVectorizer()]:
    for mod in [SVC(class_weight='balanced'), LogisticRegression(solver='liblinear', class_weight='balanced')]:
        pipe = Pipeline([
            ("feat", lang),
            ("mod", mod)
        ])
        models[get_name(lang), get_name(mod)] = pipe
        tic = time.time()
        pipe.fit(list(x_train), y_train)
        toc = time.time() 
        print(f"report for {get_name(lang), get_name(mod)}")
        train_time = toc - tic
        print(f"train time: {train_time}")
        tic = time.time()
        y_pred = pipe.predict(x_test)
        toc = time.time()
        print(f"pred time: {toc - tic}")
        d = classification_report(y_test, y_pred, output_dict=True)
        data = {
            'lang': get_name(lang), 
            'mod': get_name(mod),
            'precision': d['toxico']['precision'], 
            'recall': d['toxico']['recall'],
            'pred-time': toc - tic,
            'train-time': train_time
            
        }
        results.append(data)
        print(classification_report(y_test, y_pred))

report for ('CountVectorizer', 'SVC')
train time: 12.164035081863403
pred time: 3.6381349563598633
              precision    recall  f1-score   support

   no_toxico       0.96      0.79      0.87      2248
      toxico       0.28      0.71      0.40       252

    accuracy                           0.78      2500
   macro avg       0.62      0.75      0.63      2500
weighted avg       0.89      0.78      0.82      2500

report for ('CountVectorizer', 'LogisticRegression')
train time: 1.7707929611206055
pred time: 0.1160576343536377
              precision    recall  f1-score   support

   no_toxico       0.95      0.95      0.95      2248
      toxico       0.58      0.58      0.58       252

    accuracy                           0.91      2500
   macro avg       0.76      0.77      0.77      2500
weighted avg       0.92      0.91      0.92      2500

report for ('TfidfVectorizer', 'SVC')
train time: 16.139720916748047
pred time: 4.2620625495910645
              precision    recall 

## Entrenamiento con todos los datos

Comaparndo los resultados obtenidos para cada pipeline, utilizaremos ('CountVectorizer', 'LogisticRegression'), ya que se obtienen practicamente las mismas métricas que con ('TfidfVectorizer', 'LogisticRegression') pero con un tiempo de predicción mucho menor. Esto favorecerá que el asistente pueda determinar si el texto introducido por el usuario es tóxico en un tiempo menor.

In [10]:
x_train, x_test, y_train, y_test = train_test_split(list(df['spanish_text']), df['label'])

In [11]:
pipe = Pipeline([
        ("feat", CountVectorizer()),
        ("model", LogisticRegression(solver='liblinear', class_weight="balanced"))
    ])

In [12]:
tic = time.time()
pipe.fit(list(x_train), y_train)
toc = time.time() 
train_time = toc - tic
print(f"train time: {train_time}")
tic = time.time()
y_pred = pipe.predict(x_test)
toc = time.time()
print(f"pred time: {toc - tic}")
print(classification_report(y_test, y_pred))

train time: 21.56556510925293
pred time: 1.9688425064086914
              precision    recall  f1-score   support

   no_toxico       0.98      0.93      0.95     35798
      toxico       0.57      0.81      0.67      4095

    accuracy                           0.92     39893
   macro avg       0.77      0.87      0.81     39893
weighted avg       0.94      0.92      0.92     39893



In [13]:
joblib.dump(pipe, r'..\data\models\toxicity_model.pkl')

['..\\data\\models\\toxicity_model.pkl']

In [14]:
pipe.predict(["Lo has hecho de puta madre!"])

array(['toxico'], dtype=object)